### Read Credentials 

In [117]:
import requests, json, time, getopt, sys

# User Variables
credFile = open("..\creds\credFile.txt","r")    # one per line
                                                #--- RDP MACHINE ID---
                                                #--- LONG PASSWORD---
                                                #--- GENERATED CLIENT ID---

USERNAME = credFile.readline().rstrip('\n')
PASSWORD = credFile.readline().rstrip('\n')
CLIENT_ID = credFile.readline().rstrip('\n')

credFile.close()

print("USERNAME="+str(USERNAME))
#print("PASSWORD="+str(PASSWORD))
print("CLIENT_ID="+str(CLIENT_ID))

USERNAME=GE-A-01103867-3-3587
CLIENT_ID=4788a3857ac9449088559e215a5483890d4891fd


### Generate Token Endpoint 

In [118]:
# Application Constants
RDP_version = "/v1"
base_URL = "https://api.refinitiv.com"
category_URL = "/auth/oauth2"
endpoint_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"


TOKEN_ENDPOINT = base_URL + category_URL + RDP_version + endpoint_URL

#print("TOKEN_ENDPOINT=" + TOKEN_ENDPOINT)

In [119]:
def _requestNewToken(refreshToken):
	if refreshToken is None:
		tData = {
			"username": USERNAME,
			"password": PASSWORD,
			"grant_type": "password",
			"scope": SCOPE,
			"takeExclusiveSignOnControl": "true"
		};
	else:
		tData = {
			"refresh_token": refreshToken,
			"grant_type": "refresh_token",
		};

	# Make a REST call to get latest access token
	response = requests.post(
		TOKEN_ENDPOINT,
		headers = {
			"Accept": "application/json"
		},
		data = tData,
		auth = (
			CLIENT_ID,
			CLIENT_SECRET
		)
	)
    
	if response.status_code != 200:
		raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

	# Return the new token
	return json.loads(response.text);

In [120]:
#=============================================================================
def saveToken(tknObject):
	tf = open(TOKEN_FILE, "w+");
	print("Saving the new token");
	# Append the expiry time to token
	tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
	# Store it in the file
	json.dump(tknObject, tf, indent=4)
	
	

In [121]:
def getToken():
	try:
		print("Reading the token from: " + TOKEN_FILE);
		# Read the token from a file
		tf = open(TOKEN_FILE, "r+")
		tknObject = json.load(tf);

		# Is access token valid
		if tknObject["expiry_tm"] > time.time():
			# return access token
			return tknObject["access_token"];

		print("Token expired, refreshing a new one...");
		tf.close();
		# Get a new token from refresh token
		tknObject = _requestNewToken(tknObject["refresh_token"]);

	except Exception as exp:
		print("Caught exception: " + str(exp))
		print("Getting a new token using Password Grant...");
		tknObject = _requestNewToken(None);

	# Persist this token for future queries
	saveToken(tknObject)
	print("Token is: " + tknObject["access_token"])
	# Return access token
	return tknObject["access_token"];

### Obtain Valid Token 

In [122]:
accessToken = getToken();
print("Have token now");

Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6IkhQS2pEYUhiZmpLOVRfRHUxeGFJX1JaVmZfRjNDcHlEc0dTcDV5NWxCYVEifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwieHczVWZCblE3N25kNFQ3Tm9HRDJzdlZpTEN6WGFwQkJHRlY1MjNNSHdnTGZtWlNZbHBfbFdUeVVMRml6X0l1dU51TFBJUnRKRkFiOGItdVAzMjdaOVJ0bHJQRUROdUZsTUREam90RFRXYjhqSDkyZmowTVRIVU5pZ3RoS1djWnBJSjJtdjNNR1pTTTNCSHQxeDRBTzROdUxXdW9Ia0lUSTlnYk5aN2NfV2pYV2V5TnBhS0NvbEZsZjJiaEF1LXNHLXZNeHZwY1ktMlUtRUJOd1ExSU9Yd09PdjYteHl5WkJidU5DVm9DX2dqY1FGNV9mNnFtRkIzc0MxWHFJT2hkMnZqdVFuNGYwTTNGbXNsVndOR3ZvZEs4V2VPdVZYU2k0cVo0eGIwWlBwVVRmaXFXeEJvSFI2Zm5md1hObmJ0VFM2R0hIZ0wzRWg0eWNIVEtZWGFyX3NUUnRXYlc2QUhtRUd1QklwUmhhWnFRXy0zMkJNOWZmVFNwdG5lbVJRRGlaRS02R0NQMktBS1ZrX20xbGdJNE5YN3QzWXQ4b3JLOXMwc2MwRzE3M3FXRHVPRl9vNVRZNEhPLUxDbzFsYm9wTTUxQ0hLS212ZU1RSHlpUVpnZUgx

### Request News Metadata 

In [123]:
news_category_URL = "/data/news"
newsmeta_endpoint_URL = "/metadata"
news_param1 = "?limit=100"
NEWS_ENDPOINT = base_URL + news_category_URL + RDP_version + newsmeta_endpoint_URL 
NEWS_META_FILE = "newsMetadata.txt"

nodesWithParents = []
nodesWithoutParents = []

print("NEWS_ENDPOINT=" + NEWS_ENDPOINT)

dResp = requests.get(NEWS_ENDPOINT + news_param1 , headers = {"Authorization": "Bearer " + accessToken});

if dResp.status_code != 200:
	print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
else:
	print("Resource access successful")
	# Display data
	jResp = json.loads(dResp.text);
#	print(json.dumps(jResp, indent=2));


NEWS_ENDPOINT=https://api.refinitiv.com/data/news/v1/metadata
Resource access successful


### Request Children and Re-Categorize With Parent Information

In [124]:
def processWithChildren(dResp, jResp, parentId):
    news_param2 = "/children?limit=3000"  
    global accessToken

    if len(jResp['data']) >=200:
        print("LEN="+str(len(jResp['data'])))
    
    if dResp.status_code == 200:
#	print(json.dumps(jResp['data'], indent=2));
        for node in jResp['data']:
  #          print(node) 
            nodeIsFirstSeen = True
            if parentId != '':
                node['parentId'] = parentId 
                if node not in nodesWithParents:
                    nodesWithParents.append(node)
 #                   print("*** id= " + str(node.get('id')) + "nodesWithParents.append" )
                else :
                    nodeIsFirstSeen = False
  #                  print("DUP nodesWithParents "+node.get('id'))
            else:
                if not any(nd.get('id') == node.get('id') for nd in nodesWithParents) and node not in nodesWithoutParents:
                    nodesWithoutParents.append(node)
  #                  print("*** id= " + str(node.get('id')) + "nodesWithoutParents.append")
                else :
                    nodeIsFirstSeen = False
    #                print("DUP nodesWithoutParents "+node.get('id'))
    #        print("*** id= " + str(node.get('id')) + " *** childrenCount= " + str(node.get('childrenCount'))+ "nodeIsFirstSeen=" +str(nodeIsFirstSeen))
            if nodeIsFirstSeen == True and ((len(nodesWithParents) + len(nodesWithoutParents)) % 200) == 0:
                print("***************Inserted "+ str((len(nodesWithParents) + len(nodesWithoutParents))))
            if nodeIsFirstSeen == True and node.get('childrenCount') != 0:
                dChildrenResp = requests.get(NEWS_ENDPOINT + "/" + str(node.get('id')) + news_param2, headers = {"Authorization": "Bearer " + accessToken});

                if dChildrenResp.status_code != 200:
                    print("Unable to get children data. Code %s, Message: %s" % (dChildrenResp.status_code, dChildrenResp.text));
                    if dChildrenResp.status_code != 401:   # error other then token expired
                        break 
                    accessToken = getToken();     # token refresh on token expired
                    dChildrenResp = requests.get(NEWS_ENDPOINT + "/" + str(node.get('id')) + news_param2, headers = {"Authorization": "Bearer " + accessToken});
                else:                
                    jCResp = json.loads(dChildrenResp.text);
#                    print("about to processWithChildren")
                    processWithChildren(dChildrenResp, jCResp, node.get('id'))
#                    print(json.dumps(jCResp, indent=2));
#            else:
#                print("I AM A LEAF!")
    

In [125]:
processWithChildren(dResp, jResp,'')

Unable to get children data. Code 502, Message: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

LEN=224
***************Inserted 200
LEN=221
***************Inserted 400


### Request Next on News Metadata and (optionally) Save to File

In [126]:
#DBG nf = open(NEWS_META_FILE, "w+");
#DBG nf.write(json.dumps(jResp, indent=2))
    
#print("Next= " + jResp["meta"]["next"])

news_param2 = "?cursor=" 
while jResp["meta"]["next"]:   #not empty
    print("Next= " + jResp["meta"]["next"])
    dResp = requests.get(NEWS_ENDPOINT + news_param2 + jResp["meta"]["next"] , headers = {"Authorization": "Bearer " + accessToken});

    if dResp.status_code != 200:   #
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            break 
        accessToken = getToken();     # token refresh on token expired
        dResp = requests.get(NEWS_ENDPOINT + news_param2 + jResp["meta"]["next"] , headers = {"Authorization": "Bearer " + accessToken});
            
    print("Resource access successful")
    # Display data
    jResp = json.loads(dResp.text);
#    print(json.dumps(jResp, indent=2));
    processWithChildren(dResp, jResp,'')
        
#DBG    nf.write(json.dumps(jResp, indent=2))
#DBG nf.close()

print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<DONE child processing >>>>>>>>>>>>>>>>>>>>>>>>>>")

Next= eyJsaW1pdCI6MTAwLCJmb3J3YXJkIjp0cnVlLCJwYWdpbmF0aW9uSWQiOiIwMDAwMDAwMDk5In0=
Resource access successful
***************Inserted 600
***************Inserted 800
***************Inserted 1000
***************Inserted 1200
***************Inserted 1400
LEN=274
***************Inserted 1600
***************Inserted 1800
***************Inserted 2000
***************Inserted 2200
Next= eyJsaW1pdCI6MTAwLCJmb3J3YXJkIjp0cnVlLCJwYWdpbmF0aW9uSWQiOiIwMDAwMDAwMTk5In0=
Resource access successful
Next= eyJsaW1pdCI6MTAwLCJmb3J3YXJkIjp0cnVlLCJwYWdpbmF0aW9uSWQiOiIwMDAwMDAwMjk5In0=
Resource access successful
***************Inserted 2400
***************Inserted 2600
Next= eyJsaW1pdCI6MTAwLCJmb3J3YXJkIjp0cnVlLCJwYWdpbmF0aW9uSWQiOiIwMDAwMDAwMzk5In0=
Resource access successful
***************Inserted 2800
LEN=667
***************Inserted 3000
***************Inserted 3200
***************Inserted 3400
***************Inserted 3600
***************Inserted 3800
LEN=779
***************Inserted 4000
***************I

### Process into Tree Form

In [127]:
from anytree import Node, RenderTree

print("nodesWithoutParents length=" + str(len(nodesWithoutParents)) + "nodesWithParents length=" + str(len(nodesWithParents)))
    
for node in nodesWithoutParents:
    node['treenode'] = Node(node.get('id')) 
    
for node in nodesWithParents:
    node['treenode'] = Node(node.get('id')) 
    
for node in nodesWithParents:
    found = False
    for nWithp in nodesWithParents:
        if node.get('parentId') == nWithp.get('id'):
            node['treenode'].parent = nWithp.get('treenode')  
#            print("Node "+ str(node.get('id')) + " parent is "+ nWithp.get('id')) 
            found = True
            break
    if not found:
        for nWithoutp in nodesWithoutParents:
            if node.get('parentId') == nWithoutp.get('id'):
                node['treenode'].parent = nWithoutp.get('treenode')  
#                print("Node whose parent is parentless "+ str(node.get('id'))  + " parent is "+ nWithoutp.get('id')) # + " parent treenode is " + str(nWithoutp.get('treenode')))
                found = True
                break
    if not found:
        node['treenode'] = Node(node.get('id'))
        print("ORPHAN ?????????????????????????????????? " + node.get('id'))
        
# check for top-levels that are not really top level, just happened to be first
for index, node in enumerate(nodesWithoutParents):
    if any(nd.get('id') == node.get('id') for nd in nodesWithParents):
#        remove mislabeled top-level        
        nodesWithoutParents.remove(node) 
        print("Mislabeled empty top-level removed"+ str(node))
        
for node in nodesWithoutParents:
    print(RenderTree(node.get('treenode')))    

nodesWithoutParents length=62nodesWithParents length=12061
Mislabeled empty top-level removed{'id': 'NS:1ba169b7-7326-4951-93dd-3c4f7883987b-Source', 'label': 'Publications by Business Sectors', 'group': 'Source', 'readable': 'Source:1ba169b7-7326-4951-93dd-3c4f7883987b-Source', 'searchable': False, 'childrenCount': 635, 'treenode': Node('/NS:1ba169b7-7326-4951-93dd-3c4f7883987b-Source')}
Mislabeled empty top-level removed{'id': 'NS:THEGRD', 'label': 'The Guardian', 'group': 'Source', 'readable': 'Source:THEGRD', 'searchable': True, 'childrenCount': 0, 'treenode': Node('/NS:THEGRD')}
Mislabeled empty top-level removed{'id': 'NS:9aff4964-e93f-465b-887a-ceef8cd18745-Source', 'label': 'Business Management', 'group': 'Source', 'readable': 'Source:9aff4964-e93f-465b-887a-ceef8cd18745-Source', 'searchable': False, 'childrenCount': 36, 'treenode': Node('/NS:9aff4964-e93f-465b-887a-ceef8cd18745-Source')}
Mislabeled empty top-level removed{'id': 'NS:7fe2bbbe-4feb-44f6-b4d3-ff34bcbe334a-Source',